In [6]:
import pandas as pd

file_path = 'thirukkural_multi_language.xlsx'

tamil_df = pd.read_excel(file_path, sheet_name='Tamil')
english_df = pd.read_excel(file_path, sheet_name='English')
hindi_df = pd.read_excel(file_path, sheet_name='Only Hindi- English Format')

output_lines = []
for idx in range(len(tamil_df)):
    kural_id = tamil_df.loc[idx, 'Kural no']
    
    tamil_line = str(tamil_df.loc[idx, 'Line 1']).strip() + " " + str(tamil_df.loc[idx, 'Line 2']).strip()
    english_line = str(english_df.loc[idx, 'Translation  Line']).strip()
    hindi_line = str(hindi_df.loc[idx, 'Explanation']).strip() 

    explanation_line = str(english_df.loc[idx, 'Translation  Line']).strip() 

    text_block = f"""<kural_id>: {kural_id}
Tamil: {tamil_line}
English: {english_line}
Hindi: {hindi_line}
Explanation: {explanation_line}

"""
    output_lines.append(text_block)

with open('thirukkural_plain_text.txt', 'w', encoding='utf-8') as f:
    f.writelines(output_lines)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

model_name = "google/gemma-3-270m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

dataset = load_dataset('text', data_files={'train': 'thirukkural_train.txt'})

def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gemma3-thirukkural-sft",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="no",
    save_total_limit=2,
    learning_rate=5e-5,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator,
)

trainer.train()
